# 装饰器

1. 为函数添加额外功能
2. 不改变原函数的调用方式
3. 不改变原函数的源代码
4. 装饰器本质上是一个函数
5. 装饰器的返回值也是一个函数
6. 装饰器的使用: 使用@装饰器名

## 练习1 -- 定义 decorator
设计一个decorator，它可作用于任何函数上，并打印该函数的执行时间


In [3]:
import time, functools

# 定义装饰器
def metric(fn):
    @functools.wraps(fn)       # 装饰器语法糖，保留原函数的元信息
    def wrapper(*args, **kw):  
        start = round(time.time() * 1000)
        res = fn(*args, **kw)       # 执行一次函数, 丢掉返回值, 目的是为了计算函数执行时间
        end = round(time.time() * 1000)
        print(f'{fn.__name__} executed in {end - start} ms.')
        return res                  # 注意这里返回函数的调用结果就行了, 没必要返回 func(*args, **kw), 这样会多调用一次函数
    return wrapper



#  ------------------------------ # 
# test
@metric                # 重新包装函数
def fast(x, y):
    time.sleep(0.0012)
    return x + y

@metric                # 重新包装函数
def slow(x, y, z):
    time.sleep(0.1234)
    return x * y * z

f = fast(11, 22)       # fast 和 slow 都经过了装饰器重新包装, 执行函数时会比原函数多一些功能.
s = slow(11, 22, 33)

# 检查是否还有原函数的功能
if f != 33:
    print('测试失败!')
elif s != 7986:
    print('测试失败!')

fast executed in 1 ms.
slow executed in 129 ms.


## 练习2  -- 定义 decorator
编写一个decorator，能在函数调用的前后打印出 'begin call' 和 'end call' 的日志。

In [14]:
import time, functools

def log(func):
    # print("outside wrapper")
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print(f"begin all {func.__name__}")      # 如果这行代码写到了wrapper外面, 那么在使用 @log 的时候就会被执行, 需要注意!
        res = func(*args, **kw)                  # 调用函数
        print(f"end call {func.__name__}")
        return res                               # 返回执行结果
    return wrapper


# test
@log                # 用装饰器重新包装函数, 相当于执行一次 fast = log(fast), 此时会执行位于 log 里面同时又不在 wrapper 里的代码, 即 "print("outside wrapper")"
def fast(x, y):
    time.sleep(0.0012)
    return x + y

@log
def slow(x, y, z):
    time.sleep(0.1234)
    return x * y * z

f = fast(11, 22)       # 执行fast时, 实际上进入了 log 里面的 wrapper 函数
s = slow(11, 22, 33)


# 检查是否还有原函数的功能
if f != 33:
    print('测试失败!')
elif s != 7986:
    print('测试失败!')



begin all fast
end call fast
begin all slow
end call slow


## 练习3 -- 定义一个可以接受参数的decorator
在练习 2 的基础上, 考虑如何传递参数给decorator

In [3]:
import functools, time

def log(*text):                   # 这一层函数的作用仅仅是传递参数给装饰器, 用可变参数, 用于兼容装饰器不传递参数的情况
    def decorator(func):          # 这一层函数才是真正意义上的装饰器
        @functools.wraps(func)
        def wrapper(*args, **kw): # 这一层是原函数的执行
            print(f"{text} | begin call {func.__name__}") if text else print(f"begin call {func.__name__}")
            res = func(*args, **kw)
            print(f"{text} | end call {func.__name__}") if text else print(f"end call {func.__name__}")
            return res
        return wrapper
    return decorator


# test
@log("哈哈哈哈?")         # 用装饰器重新包装函数, 相当于执行一次 fast = log("哈哈哈哈哈?")(fast), 此时先执行 log("哈哈哈哈哈?"), 传递了参数给内部的函数 decorator; 然后执行内部的 wrapper, 打印日志, 最后才执行原来的函数
def fast(x, y):
    time.sleep(0.0012)
    return x + y

@log()                   # 这时候, 哪怕不传参, 也得带个括号, 因为log本身也是一个函数
def slow(x, y, z):
    time.sleep(0.1234)
    return x * y * z

f = fast(11, 22)     
s = slow(11, 22, 33)


# 检查是否还有原函数的功能
if f != 33:
    print('测试失败!')
elif s != 7986:
    print('测试失败!')


('哈哈哈哈?',) | begin call fast
('哈哈哈哈?',) | end call fast
begin call slow
end call slow
